In [0]:
service_principal_application_id = ""
service_principal_secret = ""                            
service_principal_tenant_id = ""

extra_configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": service_principal_application_id,
    "fs.azure.account.oauth2.client.secret": service_principal_secret,
    "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{service_principal_tenant_id}/oauth2/token",
    "fs.azure.createRemoteFileSystemDuringInitialization": "true"
}

container_name = "container1"
storage_account_name = "st24012021"
source = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/"

mount_point = "/mnt/netcoreconf"

dbutils.fs.mount(
  source = source,
  mount_point = mount_point,
  extra_configs = extra_configs)

An error occurred while calling o244.unmount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/netcoreconf; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/netcoreconf
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:128)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:68)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:576)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.unmount(DBUtilsCore.scala:587)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)
Caused by: java.lang.IllegalArgumentException: requirement failed: Directory not mounted: /mnt/netcoreconf
	at scala.Predef$.require(Predef.scala:281)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$deleteMount$1(MetadataManager.scala:453)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.$anonfun$modifyAndVerify$1(MetadataManager.scala:669)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.withRetries(MetadataManager.scala:503)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.modifyAndVerify(MetadataManager.scala:658)
	at com.databricks.backend.daemon.data.server.DefaultMetadataManager.deleteMount(MetadataManager.scala:457)
	at com.databricks.backend.daemon.data.server.handler.MountHandler.receive(MountHandler.scala:102)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1(SessionContext.scala:97)
	at com.databricks.backend.daemon.data.server.session.SessionContext.$anonfun$queryHandlers$1$adapted(SessionContext.scala:96)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at com.databricks.backend.daemon.data.server.session.SessionContext.queryHandlers(SessionContext.scala:96)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:284)
	at com.databricks.backend.daemon.data.server.DbfsServerBackend$$anonfun$receive$3.applyOrElse(DbfsServerBackend.scala:244)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$2(ServerBackend.scala:53)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend$$anonfun$commonReceive$1.applyOrElse(ServerBackend.scala:80)
	at com.databricks.rpc.ServerBackend.$anonfun$internalReceive$1(ServerBackend.scala:49)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$4(UsageLogging.scala:432)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVariable.withValue(DynamicVariable.scala:62)
	at com.databricks.logging.UsageLogging.withAttributionContext(UsageLogging.scala:235)
	at com.databricks.logging.UsageLogging.withAttributionContext$(UsageLogging.scala:232)
	at com.databricks.rpc.ServerBackend.withAttributionContext(ServerBackend.scala:16)
	at com.databricks.logging.UsageLogging.withAttributionTags(UsageLogging.scala:277)
	at com.databricks.logging.UsageLogging.withAttributionTags$(UsageLogging.scala:270)
	at com.databricks.rpc.ServerBackend.withAttributionTags(ServerBackend.scala:16)
	at com.databricks.logging.UsageLogging.rec

In [0]:
def calculate_stock_intervals_with_sql_api(spark, daily_stock):
    daily_stock.createOrReplaceTempView("my_view")

    return spark.sql("""
WITH my_cte AS
  (SELECT ROW_NUMBER() OVER (
                             ORDER BY date) AS RowNumber,
                            DATE_ADD(Date, -ROW_NUMBER() OVER (PARTITION BY StoreId, ProductId, Quantity
                                                             ORDER BY Date)) AS group,
                            StoreId, 
                            ProductId, Date, Quantity
   FROM my_view)
SELECT StoreId, 
       ProductId,
       MIN(Date) AS StartDate,
       MAX(Date) AS EndDate,
       Quantity
FROM my_cte
GROUP BY StoreId,
         ProductId,
         group,
         Quantity""")

In [0]:
df = spark.read.parquet("/mnt/netcoreconf/directory1/daily_stocks")
calculate_stock_intervals_with_sql_api(spark, df).write.parquet("/mnt/netcoreconf/directory1/stock_movements", mode="overwrite")

In [0]:
dbutils.fs.unmount(mount_point)

/mnt/netcoreconf has been unmounted.